## 0. Setup & Imports

In [44]:
# Import Libraries
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text, inspect

# Add project root to path
project_root = Path.cwd()
if 'notebooks' in str(project_root):
    project_root = project_root.parent
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Python path: {sys.path[0]}")

Project root: d:\Data_Learning\netflix-etl-pipeline
Python path: d:\Data_Learning\netflix-etl-pipeline


In [45]:
# Import custom modules
from config.config import Config
from src.extractor import NetflixExtractor
from src.transformer import NetflixTransformer
from src.loader import NetflixLoader

print("✓ All modules imported successfully")

✓ All modules imported successfully


In [46]:
# Validate Configuration
print("Database Configuration:")
print(f"  Host: {Config.DB_HOST}")
print(f"  Port: {Config.DB_PORT}")
print(f"  Database: {Config.DB_NAME}")
print(f"  User: {Config.DB_USER}")
print(f"\nData Configuration:")
print(f"  Data Path: {Config.DATA_PATH}")
print(f"  File Exists: {os.path.exists(Config.DATA_PATH)}")

Database Configuration:
  Host: localhost
  Port: 5432
  Database: netflix_db
  User: netflix_user

Data Configuration:
  Data Path: ./data/netflix_titles.csv
  File Exists: True


---

## 1. EXTRACT - Trích xuất Dữ liệu

In [47]:
# Initialize Extractor with data path
data_path = Path(Config.DATA_PATH)
print(f"Data path: {data_path}")
print(f"File exists: {os.path.exists(data_path)}")

extractor = NetflixExtractor(data_path=str(data_path))
print("✓ Extractor initialized")

Data path: data\netflix_titles.csv
File exists: True
✓ Extractor initialized


In [ ]:
# Check if data exists, if not download automatically from Kaggle
data_file = Config.DATA_PATH

if not os.path.exists(data_file):
    print("Netflix dataset not found. Attempting to download from Kaggle...")
    print(f"Download destination: {os.path.abspath(data_file)}\n")
    
    try:
        # Try using Kaggle API to download
        from kaggle.api.kaggle_api_extended import KaggleApi
        
        # Create data directory if it doesn't exist
        os.makedirs(os.path.dirname(data_file), exist_ok=True)
        
        print("Initializing Kaggle API...")
        api = KaggleApi()
        api.authenticate()
        
        print("Downloading dataset: shivamb/netflix-shows")
        api.dataset_download_files("shivamb/netflix-shows", path=os.path.dirname(data_file), unzip=True)
        print("✓ Dataset downloaded successfully!\n")
        
    except Exception as e:
        print(f"✗ Kaggle download failed: {str(e)}\n")
        print("📥 Please download manually:")
        print("  1. Visit: https://www.kaggle.com/datasets/shivamb/netflix-shows")
        print("  2. Click 'Download'")
        print(f"  3. Extract netflix_titles.csv to: {os.path.dirname(data_file)}/")
        raise
else:
    print(f"✓ Dataset found at: {os.path.abspath(data_file)}")

# Extract data from CSV
print("\nLoading data from CSV...")
df_raw = extractor.extract_from_csv()
print("\n✓ Data loaded successfully!")

✓ Dataset found at: d:\Data_Learning\netflix-etl-pipeline\notebooks\data\netflix_titles.csv

Loading data from CSV...
Reading data from data\netflix_titles.csv...
✓ Extracted 8807 rows and 12 columns
Columns: ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']

✓ Data loaded successfully!


In [49]:
# Display data information
extractor.get_data_info(df_raw)


DATA INFORMATION
Shape: (8807, 12)

Column Names and Types:
show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

Missing Values:
show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

First few rows:
  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factor

In [50]:
# Validate data
is_valid = extractor.validate_data(df_raw)
if is_valid:
    print("✓ Data validation passed")
else:
    print("✗ Data validation failed")

✓ All required columns present
✓ Data validation passed


---

## 2. TRANSFORM - Chuyển đổi Dữ liệu

In [51]:
# Reload modules to get latest changes
import importlib
import sys

# Remove cached modules
modules_to_reload = [m for m in sys.modules.keys() if m.startswith('src.') or m.startswith('config.')]
for m in modules_to_reload:
    del sys.modules[m]

# Re-import fresh modules
from config.config import Config
from src.extractor import NetflixExtractor
from src.transformer import NetflixTransformer
from src.loader import NetflixLoader

# Reinitialize transformer with fresh data
transformer = NetflixTransformer(df_raw)
print("✓ Transformer reinitialized with fresh modules")

✓ Transformer reinitialized with fresh modules


In [52]:
# Execute full transformation pipeline
star_schema = transformer.transform()


NETFLIX DATA TRANSFORMATION PIPELINE

STEP 1: CLEANING DATA

Missing values before cleaning:
  director: 2634 (29.91%)
  country: 831 (9.44%)
  date_added: 10 (0.11%)
  rating: 4 (0.05%)

Rows removed: 3057
Rows remaining: 5750
✓ Data cleaning completed

STEP 2: NORMALIZING DATES
⚠ Warning: 5 invalid dates found
✓ Date normalization completed
Sample dates: ['2021-09-25' '2021-09-24' '2021-09-24']

STEP 3: NORMALIZING TEXT
✓ Normalized text in column: director
✓ Normalized text in column: country
✓ Normalized text in column: listed_in
✓ Normalized text in column: title

STEP 4: EXPLODING GENRES
Rows before explode: 5750Rows before explode: 5750


Rows after explode: 12567
Unique genres: 42
✓ Genre explosion completed

STEP 5: CREATING STAR SCHEMA

1. Creating dim_genres...
   ✓ Created 42 unique genres

2. Creating dim_movies...
   ✓ Created 5750 unique movies

3. Creating movies_genres junction table...
   ✓ Created 12567 movie-genre relationships

--------------------------------------------------
STAR SCHEMA SUMMARY:
  dim_movies: 5750 rows
  dim_genres: 42 rows
  movies_genres: 12567 rows
--------------------------------------------------

✓ TRANSFORMATION COMPLETED SUCCESSFULLY



In [53]:
# Extract dataframes from star schema
dim_movies = star_schema['dim_movies']
dim_genres = star_schema['dim_genres']
movies_genres = star_schema['movies_genres']

print(f"\ndim_movies shape: {dim_movies.shape}")
print(f"dim_genres shape: {dim_genres.shape}")
print(f"movies_genres shape: {movies_genres.shape}")


dim_movies shape: (5750, 10)
dim_genres shape: (42, 2)
movies_genres shape: (12567, 2)


In [54]:
# Display dim_movies sample
print("\ndim_movies (first 5 rows):")
print(dim_movies.head())
print(f"\nColumns: {dim_movies.columns.tolist()}")


dim_movies (first 5 rows):
   movie_id                          title     type             director  \
0         1           Dick Johnson Is Dead    Movie      Kirsten Johnson   
1         2                        Sankofa    Movie         Haile Gerima   
2         3  The Great British Baking Show  TV Show      Andy Devonshire   
3         4                   The Starling    Movie       Theodore Melfi   
4         5                   Je Suis Karl    Movie  Christian Schwochow   

                                             country  date_added  \
0                                      United States  2021-09-25   
1  United States, Ghana, Burkina Faso, United Kin...  2021-09-24   
2                                     United Kingdom  2021-09-24   
3                                      United States  2021-09-24   
4                            Germany, Czech Republic  2021-09-23   

   release_year rating   duration  \
0          2020  PG-13     90 min   
1          1993  TV-MA    125 mi

In [55]:
# Display dim_genres sample
print("\ndim_genres (first 15 rows):")
print(dim_genres.head(15))


dim_genres (first 15 rows):
    genre_id                genre_name
0          1             Documentaries
1          2                    Dramas
2          3        Independent Movies
3          4      International Movies
4          5          British TV Shows
5          6                Reality TV
6          7                  Comedies
7          8           Romantic Movies
8          9             Horror Movies
9         10          Sci-Fi & Fantasy
10        11                 Thrillers
11        12        Action & Adventure
12        13            Classic Movies
13        14            Anime Features
14        15  Children & Family Movies


In [56]:
# Display movies_genres sample
print("\nmovies_genres (first 10 rows):")
print(movies_genres.head(10))


movies_genres (first 10 rows):
   movie_id  genre_id
0         1         1
1         2         2
2         2         3
3         2         4
4         3         5
5         3         6
6         4         7
7         4         2
8         5         2
9         5         4


In [57]:
# Data Quality Checks
print("\n" + "="*50)
print("DATA QUALITY CHECKS")
print("="*50)

# Check for duplicates
print(f"\n1. Duplicate check:")
print(f"   dim_movies duplicates: {dim_movies.duplicated().sum()}")
print(f"   dim_genres duplicates: {dim_genres.duplicated().sum()}")
print(f"   movies_genres duplicates: {movies_genres.duplicated().sum()}")

# Check for NULL values
print(f"\n2. NULL values:")
print(f"   dim_movies: {dim_movies.isnull().sum().sum()}")
print(f"   dim_genres: {dim_genres.isnull().sum().sum()}")
print(f"   movies_genres: {movies_genres.isnull().sum().sum()}")

# Check unique values
print(f"\n3. Unique values:")
print(f"   Unique movies: {dim_movies['movie_id'].nunique()}")
print(f"   Unique genres: {dim_genres['genre_id'].nunique()}")
print(f"   Movie types: {dim_movies['type'].unique()}")
print(f"   Rating categories: {dim_movies['rating'].nunique()}")

print("\n✓ All quality checks completed")


DATA QUALITY CHECKS

1. Duplicate check:
   dim_movies duplicates: 0
   dim_genres duplicates: 0
   movies_genres duplicates: 0

2. NULL values:
   dim_movies: 8
   dim_genres: 0
   movies_genres: 0

3. Unique values:
   Unique movies: 5750
   Unique genres: 42
   Movie types: ['Movie' 'TV Show']
   Rating categories: 17

✓ All quality checks completed


---

## 3. LOAD - Tải Dữ liệu vào PostgreSQL

In [58]:
# Initialize Loader
loader = NetflixLoader()
print("✓ Loader initialized")

✓ Loader initialized


In [59]:
# Connect to PostgreSQL
try:
    loader.connect()
    print("✓ Connection successful")
except Exception as e:
    print(f"✗ Connection failed: {str(e)}")
    print("\nMake sure PostgreSQL is running:")
    print("  docker-compose up -d")
    raise

Connecting to PostgreSQL...
✓ Connected to netflix_db @ localhost:5432
✓ Connection successful
✓ Connected to netflix_db @ localhost:5432
✓ Connection successful


In [60]:
# Load data to PostgreSQL
try:
    load_results = loader.load_all(star_schema)
    print("\n✓ All tables loaded successfully")
except Exception as e:
    print(f"✗ Load failed: {str(e)}")
    raise


LOADING DATA TO POSTGRESQL

--------------------------------------------------
Loading dim_genres...
--------------------------------------------------
✓ Loaded 42 genres

--------------------------------------------------
Loading dim_movies...
--------------------------------------------------
✓ Loaded 750 movies

--------------------------------------------------
Loading movies_genres...
--------------------------------------------------
✓ Loaded 750 movies

--------------------------------------------------
Loading movies_genres...
--------------------------------------------------
✓ Loaded 567 movie-genre relationships

--------------------------------------------------
LOAD SUMMARY:
  dim_genres: 42 rows
  dim_movies: 750 rows
  movies_genres: 567 rows
--------------------------------------------------

✓ All tables loaded successfully
✓ Loaded 567 movie-genre relationships

--------------------------------------------------
LOAD SUMMARY:
  dim_genres: 42 rows
  dim_movies: 750 r

In [61]:
# Validate loaded data
try:
    validation = loader.validate_load()
    print("\n✓ Data validation completed")
except Exception as e:
    print(f"✗ Validation failed: {str(e)}")
    raise


VALIDATING LOADED DATA
dim_movies: 5750 rows
dim_genres: 42 rows
movies_genres: 12567 rows

Sample movies:
   movie_id                          title     type  release_year
0         1           Dick Johnson Is Dead    Movie          2020
1         2                        Sankofa    Movie          1993
2         3  The Great British Baking Show  TV Show          2021
3         4                   The Starling    Movie          2021
4         5                   Je Suis Karl    Movie          2021

Sample genres:
   genre_id            genre_name
0         1         Documentaries
1         2                Dramas
2         3    Independent Movies
3         4  International Movies
4         5      British TV Shows
5         6            Reality TV
6         7              Comedies
7         8       Romantic Movies
8         9         Horror Movies
9        10      Sci-Fi & Fantasy

✓ Data validation completed


In [62]:
# Disconnect from database
loader.disconnect()

✓ Disconnected from database


---

## 4. POST-LOAD Analysis - Truy vấn Dữ liệu

In [63]:
# Reconnect for analysis
loader2 = NetflixLoader()
engine = loader2.connect()

Connecting to PostgreSQL...
✓ Connected to netflix_db @ localhost:5432


In [64]:
# Query 1: Movie vs TV Show counts
query1 = """
SELECT type, COUNT(*) as count
FROM dim_movies
GROUP BY type
ORDER BY count DESC;
"""

result1 = pd.read_sql(query1, engine)
print("\n1. Movies vs TV Shows:")
print(result1.to_string(index=False))


1. Movies vs TV Shows:
   type  count
  Movie   5561
TV Show    189


In [65]:
# Query 2: Top 15 genres
query2 = """
SELECT dg.genre_name, COUNT(mg.movie_id) as movie_count
FROM dim_genres dg
LEFT JOIN movies_genres mg ON dg.genre_id = mg.genre_id
GROUP BY dg.genre_id, dg.genre_name
ORDER BY movie_count DESC
LIMIT 15;
"""

result2 = pd.read_sql(query2, engine)
print("\n2. Top 15 Genres:")
print(result2.to_string(index=False))


2. Top 15 Genres:
              genre_name  movie_count
    International Movies         2505
                  Dramas         2297
                Comedies         1555
      Action & Adventure          809
           Documentaries          751
      Independent Movies          740
         Romantic Movies          580
               Thrillers          547
Children & Family Movies          513
           Horror Movies          338
        Music & Musicals          318
         Stand-Up Comedy          294
        Sci-Fi & Fantasy          239
           Sports Movies          197
          Classic Movies          115


In [66]:
# Query 3: Movies by release year
query3 = """
SELECT release_year, COUNT(*) as count
FROM dim_movies
GROUP BY release_year
ORDER BY release_year DESC
LIMIT 20;
"""

result3 = pd.read_sql(query3, engine)
print("\n3. Top 20 Release Years:")
print(result3.to_string(index=False))


3. Top 20 Release Years:
 release_year  count
         2021    185
         2020    488
         2019    571
         2018    707
         2017    736
         2016    640
         2015    379
         2014    257
         2013    213
         2012    169
         2011    138
         2010    146
         2009    112
         2008    110
         2007     74
         2006     85
         2005     66
         2004     54
         2003     50
         2002     44


In [67]:
# Query 4: Content ratings distribution
query4 = """
SELECT rating, COUNT(*) as count
FROM dim_movies
GROUP BY rating
ORDER BY count DESC;
"""

result4 = pd.read_sql(query4, engine)
print("\n4. Content Ratings Distribution:")
print(result4.to_string(index=False))


4. Content Ratings Distribution:
  rating  count
   TV-MA   1979
   TV-14   1330
       R    784
   TV-PG    505
   PG-13    481
      PG    281
    TV-G    103
   TV-Y7     81
    TV-Y     79
      NR     75
       G     41
TV-Y7-FV      3
      UR      3
   NC-17      2
  66 min      1
  84 min      1
  74 min      1


In [68]:
# Query 5: Movies by country (top 20)
query5 = """
SELECT country, COUNT(*) as count
FROM dim_movies
WHERE country IS NOT NULL
GROUP BY country
ORDER BY count DESC
LIMIT 20;
"""

result5 = pd.read_sql(query5, engine)
print("\n5. Top 20 Countries:")
print(result5.to_string(index=False))


5. Top 20 Countries:
                      country  count
                United States   2048
                        India    890
               United Kingdom    212
                       Canada    119
                        Spain    106
                        Egypt     90
                      Nigeria     90
                        Japan     85
                       France     81
                       Turkey     79
                    Indonesia     77
                  Philippines     74
                       Mexico     71
United Kingdom, United States     61
                       Brazil     53
        United States, Canada     52
                    Hong Kong     49
                  South Korea     49
                      Germany     45
                     Thailand     42


In [69]:
# Disconnect
loader2.disconnect()

✓ Disconnected from database


---

## ✓ ETL Pipeline Completed Successfully!

Tất cả dữ liệu đã được:
- ✓ Trích xuất từ CSV
- ✓ Làm sạch và chuẩn hóa
- ✓ Tách thể loại (explode)
- ✓ Mô hình hóa theo Star Schema
- ✓ Tải vào PostgreSQL

### Thống kê Cuối cùng
- **Tổng Phim/TV Shows:** Xem Query 1
- **Tổng Thể loại:** Xem Query 2
- **Khoảng thời gian:** 1920 - 2021
- **Xếp hạn:** PG, TV-14, TV-MA, R, PG-13, ...
- **Quốc gia chính:** United States, India, Canada, ...

---

**Ngày cập nhật:** November 16, 2025